In [1]:
import pandas as pd

# Load the processed CSV file
csv_file_path = "processed_cricket_data.csv"  # Ensure this path matches the file location
crick_df = pd.read_csv(csv_file_path)

# Use crick_df for creating visualizations
print(crick_df.head())  # Optional: Check the loaded data


   Unnamed: 0.1  Unnamed: 0        date          venue match_category team_1  \
0             0           3  2011-02-21      Ahmedabad   League-Match    AUS   
1             1          48  2011-04-02       Wankhede          Final     SL   
2             2          28  2011-03-12         Nagpur   League-Match    IND   
3             3          40  2011-03-20   Eden Gardens   League-Match    ZIM   
4             6          42  2011-03-23         Mirpur          Final     WI   

  team_2  team_1_runs  team_1_wickets  team_2_runs  ...  match_status  \
0    ZIM        262.0             6.0        171.0  ...        played   
1    IND        274.0             6.0        277.0  ...        played   
2     SA        296.0             0.0        300.0  ...        played   
3  KENYA        308.0             6.0        147.0  ...        played   
4    PAK        112.0             0.0        113.0  ...        played   

    winning_team  best_bowler_1 best_bowler_1_wick    best_bowler_2  \
0     Aus

In [2]:
#Load the Data

csv_file_path = "./processed_cricket_data.csv"
crick_df = pd.read_csv(csv_file_path)